In [1]:
require 'torch' -- light my fire

Now, with some Lua under our belts (space suits?) we can get on to getting on to the fun part of designing and training neural networks! Before we can do that, though, we need to learn an array of things about Torch arrays!

If you already use a scientific computing platform like SciPy, Julia, R, or [heaven forbid] MATLAB, you'll probably agree that the n-dimensional array is probably the most imporant feature! Torch is no different. Introducing the `Tensor`:

# Tensors

The aptly-named [Tensor](https://torch7.readthedocs.io/en/latest/tensor/index.html) is Torch's multi-dimensional array data structure which can be created, indexed, sliced, resized, and otherwise modified to suit all of your mathematical needs. These are distinct from tables and are significantly faster and more compact.

There's a caveat, however: a Tensor is just a *view* of some bytes somewhere in memory, or the Tensor's `Storage`. Practically, this means is that operations like slicing, indexing, and reshaping are cheap but allocation and resizing are not! This will be important to keep in mind when working on the GPU as allocations are slow and can leak memory.

## Creating Tensors

The simplest and most generic way of getting a Tensor is to use `torch.Tensor()`. Passing in sizes will automatically allocate (but not initialize!) a `Storage`.

In [2]:
torch.Tensor(2, 3, 4)

(1,.,.) = 
  6.9520e-310  7.4110e-322   0.0000e+00   0.0000e+00
  6.9519e-310  6.9641e+252   8.0341e-95  3.1454e+161
  9.9412e-143  5.0462e+180  7.4978e+247  3.8863e+285

(2,.,.) = 
  5.3799e+151  1.6775e+243  4.5481e-144  2.6100e+251
  6.1679e+223   1.9657e-62  7.9840e+159  8.0929e+175
  5.5622e+180  3.5102e+151  1.3042e-142   1.4427e-71
[torch.DoubleTensor of size 2x3x4]



If you're so inclined, you can create a Tensor filled with zeros or ones by using `torch.zeros()` and `torch.ones()`, respectively. Additionally, you can pass in your own data:

In [3]:
torch.Tensor{{1, 2, 3}, {4, 5, 6}, {7, 8, 9}}

 1  2  3
 4  5  6
 7  8  9
[torch.DoubleTensor of size 3x3]



## Tensor types

"Wait a second" you say. "What's this bit about `torch.DoubleTensor`?"

Another great question! That's the *type* of the Tensor which determines how the rest of Torch interprets the data contained by the `Storage`. Here are some more of the Tensor types that you should know:

* `torch.FloatTensor`: 32-bit floating-point (useful for data sent to/received from a 32-bit floating-point GPU)
* `torch.LongTensor`: 64-bit integers (useful for holding array indices)
* `torch.ByteTensor`: 8-bit unsigned numbers (useful for holding binary masks)
* `torch.CudaTensor`: 32-bit fp numbers that have their storage allocated on the GPU
* `torch.CudaHalfTensor`: new-fangled type that uses half the memory of fp32 and is faster on some GPUs (e.g., Pascal)

Don't concern yourself too much, yet, with the fancy CUDA types. Since we don't have GPUs for this workshop, we won't need them, but they're virtually indispensable when training larger, more complicated models. Figuring out how the CUDA backend works has a slight (read: quite steep) learning curve, so there'll be a guide on this later.

The main idea is that certain operations require certain Tensor types; you always convert a Tensor and its Storage to a different type (allocation warning!) by calling `theTensor:<type>()`

In [4]:
print(torch.rand(2, 3):mul(42):long()) -- converts using floor(x)

 26  32   5
 30  14  38
[torch.LongTensor of size 2x3]



## It's My Data and I Want it NOW! (View A.G. Tensworth 877...)

When we're not crashing dense matrices in multiplications, we usually only want to deal with small segments of our data. For this, all we must do is create new Tensors that only look at a portion of the original Storage.

To help introduce the next concepts, we'll need a volunteer:

In [5]:
friend = torch.Tensor(3, 4):range(1, 12)
print(friend, friend:size())

  1   2   3   4
  5   6   7   8
  9  10  11  12
[torch.DoubleTensor of size 3x4]

 3
 4
[torch.LongStorage of size 2]



### Slicing and Indexing

In [6]:
row1 = friend[3]
col2 = friend[{{}, 2}]
eight = friend[{2, 4}]
sevenEleven = friend[{{2,3}, 3}]
print(sevenEleven)
assert(friend:size(1) == 3 and friend:size(2) == 4) -- friend remains unchanged

  7
 11
[torch.DoubleTensor of size 2]



If you're coming from one of the other scientific computing platforms, you'll notice the similarity between an inner table and the `:` operator.

While specifying indices using tables is convenient and all, sometimes it's easier to directly request the data you want:

In [7]:
row1 = friend:select(1, 1) -- select(dimension, index along that dimension)
col2 = friend:select(2, 2)
sevenEleven = friend:sub(2, 3, 3, 3) -- sub(from1, to1, from2, to2, ...)
alsoSevenEleven = friend:narrow(1, 2, 2):narrow(2, 3, 1) -- narrow(dimension, startIdx, sliceLen)
assert(sevenEleven:equal(alsoSevenEleven))

### Assignment

Not bad! Now that we can tell Torch which indices we want, we can start to get choosy with their contents.

When using the convenient table syntax, the `=` operator is overloaded to do assignment. If, instead, you go the route of `select`, `narrow`, and `sub`, you'll have to use the likes of `copy` and `fill`.

In [8]:
oldFriend = friend:clone() -- forgotten but not gone
friend[{1, 1}] = 0
print(friend)

  0   2   3   4
  5   6   7   8
  9  10  11  12
[torch.DoubleTensor of size 3x4]



In [9]:
friend:select(1, 3):copy(oldFriend[1]):mul(-1)
print(friend)

 0  2  3  4
 5  6  7  8
-1 -2 -3 -4
[torch.DoubleTensor of size 3x4]



In [10]:
friend:narrow(2, 2, 2):select(1, 2):zero()
print(friend)

 0  2  3  4
 5  0  0  8
-1 -2 -3 -4
[torch.DoubleTensor of size 3x4]



Presented above was only a selected subset (no pun intended) of the full Tensor indexing API. You will want to familiarize yourself with [the rest of the commands](https://torch7.readthedocs.io/en/latest/tensor/index.html) in order to write maximally terse and performant code!

## Tensor Lifecycle

When training neural networks on the GPU, the life of a Tensor looks a bit like this

1. Create a Tensor in RAM
2. Fill the Tensor with a batch of data
3. Copy the Tensor to a pre-allocated counterpart on the GPU
4. Trash the RAM Tensor

It's kind of like a bucket brigade except with numbers for water, GPUs for burning buildings, and math for fire!
"Gee, I can't imagine how lit this guy must have been when he came up with that one..." you think to yourself.  
Well, let's just say that I have a few models training while I'm writing this.  
I bet you're really confused now!

# 🎉 More Congratulations! 🎉

Hooray! You now know how to make pretty, custom Tensors.

We totally skipped over all of the math stuff you can do with Tensors, but we'll be doing that in the next part, the best part, the *net* part!